<a href="https://colab.research.google.com/github/leadeeerq/data-science/blob/main/230323_PySpark2ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 5.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=0f2c52afc2507d454b3b33603b2938289c9e0c326037459f10685f1dd8119fba
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
import pyspark
import pandas as pd

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practise').getOrCreate()

In [6]:
training = spark.read.csv('/content/drive/MyDrive/Projekty Data Science/PySpark/test1.csv', header = True, inferSchema = True)

In [7]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [8]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
training.columns

['Name', 'age', 'Experience', 'Salary']

##Vectorassembler - a feature transformer that merges multiple columns into a vector column.

In [12]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ['age', 'Experience'], outputCol = 'Independent feature')

In [13]:
output = featureassembler.transform(training)

In [14]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [15]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent feature']

In [16]:
finalized_data = output.select('Independent feature', 'Salary')

In [17]:
finalized_data.show()

+-------------------+------+
|Independent feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



##Linear regression

In [38]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = 'Independent feature', labelCol = 'Salary')
regressor = regressor.fit(train_data)

In [39]:
train_data.show()

+-------------------+------+
|Independent feature|Salary|
+-------------------+------+
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
|         [29.0,4.0]| 20000|
|         [30.0,8.0]| 25000|
|        [31.0,10.0]| 30000|
+-------------------+------+



In [40]:
test_data.show()

+-------------------+------+
|Independent feature|Salary|
+-------------------+------+
|         [24.0,3.0]| 20000|
+-------------------+------+



In [41]:
regressor.coefficients

DenseVector([-64.8464, 1584.7554])

In [42]:
regressor.intercept

15414.10693970355

In [43]:
pred_results = regressor.evaluate(test_data)

In [44]:
pred_results.predictions.show()

+-------------------+------+------------------+
|Independent feature|Salary|        prediction|
+-------------------+------+------------------+
|         [24.0,3.0]| 20000|18612.059158134216|
+-------------------+------+------------------+



In [45]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1387.940841865784, 1926379.7805191013)